<a href="https://colab.research.google.com/github/navyaravi/INFO_5731_Spring_2021/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

**Installing Required Libraries**

In [1]:
!pip install polyglot

     |████████████████████████████████| 133kB 5.2MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=85879a4dd54c34371be9c195cf9ca58961d1a272e83040ae6976f4aa7963bd4a
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot


In [2]:
!pip install pyicu

     |████████████████████████████████| 235kB 5.4MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306389 sha256=1f5a3e656fe497a75276816ff768bb624039841e331ab15ea6c3635ce2e3ec6e
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu


In [3]:
!pip install pycld2

     |████████████████████████████████| 41.4MB 117kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834267 sha256=92abac40a1f94305ff769bfbf39e786051754338547765e22f9553e22430155c
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [4]:
!pip install morfessor

In [5]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

**Reading Data**

In [30]:
# Write your code here

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/navyaravi/INFO_5731_Spring_2021/main/Sentiment_Analysis_Reviews.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

**Basic Preprocessing**

In [9]:
from nltk.corpus import stopwords
from textblob import TextBlob
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Review'] = df['Review'].apply(lambda x: str(TextBlob(x).correct()))

**Extracting Words Related to Sentiment using Polyglot**

In [10]:
from polyglot.text import Text
sentimental_words = []
for line in df['Review']:
  text = Text(line)
  for w in text.words:
    if w.polarity != 0:
      sentimental_words.append(w)
sentimental_words

['brilliance',
 'grotesque',
 'hard',
 'mesmerizing',
 'tragic',
 'funny',
 'rollercoaster',
 'proper',
 'best',
 'isolated',
 'sorry',
 'encourage',
 'better',
 'respect',
 'like',
 'isolated',
 'masterpiece',
 'best',
 'best',
 'best',
 'willingly',
 'remarkable',
 'disturbing',
 'violent',
 'proving',
 'corruption',
 'rich',
 'talented',
 'believable',
 'shake',
 'bitter',
 'comforting',
 'perfect',
 'fearless',
 'stunning',
 'impossible',
 'dark',
 'joke',
 'stunned',
 'accomplished',
 'impossible',
 'exceeds',
 'dark',
 'crime',
 'like',
 'falling',
 'joke',
 'happy',
 'ignore',
 'complaints',
 'pernicious',
 'embarrassing',
 'better',
 'criminal',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'romantic',
 'scars',
 'amazing',
 'best',
 'breathtaking',
 'uncomfortable',
 'like',
 'horrible',
 'great',
 'inspiration',
 'honest',
 'violent',
 'disturbing',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'madness',
 'enjoy',
 'masterpiece',
 'amazing',
 'enjoy',
 'jo

**Ranking Sentiment Words By Frequency**

In [11]:
from collections import Counter
frequencies = Counter(sentimental_words)
rank_df = pd.DataFrame(list(frequencies.most_common()), columns=['Words', 'Frequencies'])
rank_df.index = list(range(1, len(frequencies.most_common())+1))
rank_df

,Words,Frequencies
1,joke,147
2,like,54
3,good,54
4,best,30
5,great,28
...,...,...
507,advantage,1
508,disappointing,1
509,creep,1
510,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [57]:
def Encoder(senti):
  sentiment_encoded = 0
  if senti == 'Positive':
    sentiment_encoded = 1
  elif senti == 'Negative':
    sentiment_encoded = 2
  return sentiment_encoded

In [58]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/navyaravi/INFO_5731_Spring_2021/main/Sentiment_Analysis_Reviews.csv')
#df['Sentiment Analysis'] = df['Review'].apply(lambda x: le.fit_transform(astype(str)), axi=0, result_type='expand')


In [59]:
actual_sentiment = []
for senti in df['Sentiment Analysis']:
  actual_sentiment.append(Encoder(senti))
  
  #dataset_train.iloc[:,i] = dataset_train.apply(lambda i:le.fit_transform(dataset_train[i].astype(str)), axis=0, result_type='expand')

**Sentimental Analysis usig Text Blob**

In [60]:
textblob_sentiments = []
for line in df['Review']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_sentiments.append(1)
  elif polarity < 0:
    textblob_sentiments.append(2)
  elif polarity == 0.0:
    textblob_sentiments.append(0)
text_blob_df = pd.DataFrame(list(zip(df['Review'], actual_sentiment, textblob_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
text_blob_df

,Reviews,Actual,Predicted
0,"Every once in a while a movie comes, that trul...",1,2
1,This is a movie that only those who have felt ...,1,2
2,"Truly a masterpiece, The Best Hollywood film o...",1,1
3,Joaquin Phoenix gives a tour de force performa...,1,2
4,Most of the time movies are anticipated like t...,0,1
...,...,...,...
101,"One of the worst movies I have ever seen. Ok, ...",2,1
102,I didnt really know much about this movie. I h...,1,1
103,Film was directed and produced poorly. Opportu...,2,2
104,"Wow, what a movie! I have to admit, When I fir...",1,1


In [61]:

from sklearn.metrics import f1_score, accuracy_score
text_blob_accuracy = accuracy_score(text_blob_df['Actual'], text_blob_df['Predicted'])*100
text_blob_f1_score = f1_score(text_blob_df['Actual'], text_blob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(text_blob_accuracy, text_blob_f1_score))

Text Blob Accuracy is 57.54716981132076 and F1 Score is 0.4672796654893842


**Sentimental Analysis using Vader**

In [62]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [63]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_sentiments = []
for line in df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_sentiments.append(1)
  elif polarity < 0:
    vader_sentiments.append(2)
  elif polarity == 0.0:
    vader_sentiments.append(0)
vader_df = pd.DataFrame(list(zip(df['Review'], actual_sentiment, vader_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

,Reviews,Actual,Predicted
0,"Every once in a while a movie comes, that trul...",1,1
1,This is a movie that only those who have felt ...,1,1
2,"Truly a masterpiece, The Best Hollywood film o...",1,1
3,Joaquin Phoenix gives a tour de force performa...,1,1
4,Most of the time movies are anticipated like t...,0,2
...,...,...,...
101,"One of the worst movies I have ever seen. Ok, ...",2,2
102,I didnt really know much about this movie. I h...,1,1
103,Film was directed and produced poorly. Opportu...,2,2
104,"Wow, what a movie! I have to admit, When I fir...",1,1


In [64]:
from sklearn.metrics import f1_score, accuracy_score
vader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
vader_f1_score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(vader_accuracy, vader_f1_score))

Vader Accuracy is 56.60377358490566 and F1 Score is 0.41930141523637454


**Sentimenatl Analysis using SVM**

In [65]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [80]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

df["Review"] = df["Review"].replace(np.nan, 'none', regex=True)
df["Sentiment Analysis"] = df["Sentiment Analysis"].replace(np.nan, 'none', regex=True)

Train_X, Test_X, Train_Y, Test_Y = train_test_split(df['Review'],df['Sentiment Analysis'],test_size=0.33)
Encoder = LabelEncoder()
df.Review = Encoder.fit_transform(df.Review.values)
Train_X = [str (item) for item in Train_X]

Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
#Train_X_vectors = TFid_vectorizer.fit_transform(Train_X)
#Test_X_vectors = TFid_vectorizer.transform(Test_X)

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_X_vectors, Train_Y)
predicted = svm_model.predict(Test_X_vectors)
report = classification_report(Test_Y, predicted, output_dict=True)
report